In [1]:
import tensorflow as tf
import numpy as np
import random
import os

In [2]:
# Leitura do arquivo de texto
text = open('luladiscursos.txt', "r", encoding="utf-8").read()

In [3]:
# Converte o texto em um tensor de caracteres
chars = tf.strings.unicode_decode(text, input_encoding='UTF-8')

In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

96 unique characters


In [5]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [6]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [7]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[50, 51, 52, 53, 54, 55, 56], [73, 74, 75]]>

In [8]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [9]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [10]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [11]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [12]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(103855,), dtype=int64, numpy=array([ 1, 28, 58, ..., 50, 53, 64], dtype=int64)>

In [13]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [14]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))



D
i
s
c
u
r
s
o
 


In [15]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [16]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\n' b'D' b'i' b's' b'c' b'u' b'r' b's' b'o' b' ' b'1' b':' b' ' b'\n'
 b'Q' b'u' b'e' b'r' b'o' b' ' b'r' b'e' b'i' b't' b'e' b'r' b'a' b'r'
 b' ' b'a' b' ' b'i' b'm' b'p' b'o' b'r' b't' b'\xc3\xa2' b'n' b'c' b'i'
 b'a' b' ' b'd' b'a' b' ' b'm' b'u' b'd' b'a' b'n' b'\xc3\xa7' b'a' b' '
 b'p' b'a' b'r' b'a' b' ' b't' b'o' b'd' b'o' b's' b' ' b'o' b's' b' '
 b'c' b'i' b'd' b'a' b'd' b'\xc3\xa3' b'o' b's' b' ' b'd' b'o' b' ' b'B'
 b'r' b'a' b's' b'i' b'l' b'.' b' ' b'\xc3\x89' b' ' b'h' b'o' b'r' b'a'
 b' ' b'd' b'e' b' ' b't' b'r' b'a'], shape=(101,), dtype=string)


In [17]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'\nDiscurso 1: \nQuero reiterar a import\xc3\xa2ncia da mudan\xc3\xa7a para todos os cidad\xc3\xa3os do Brasil. \xc3\x89 hora de tra'
b'nsformar o Brasil em uma na\xc3\xa7\xc3\xa3o soberana e justa. Mudaremos com coragem e humildade, cientes de que \xc3\xa9 '
b'um processo gradual, baseado no di\xc3\xa1logo e na negocia\xc3\xa7\xc3\xa3o.\n\nO Brasil \xc3\xa9 complexo, com quase 175 milh\xc3\xb5es '
b'de habitantes, e n\xc3\xa3o podemos permitir que ele siga sem um verdadeiro projeto de desenvolvimento nacio'
b'nal. Precisamos exercer a paci\xc3\xaancia e a perseveran\xc3\xa7a, sem atropelos, plantando \xc3\xa1rvores antes de colhe'


In [18]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [19]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [20]:
dataset = sequences.map(split_input_target)

In [21]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'\nDiscurso 1: \nQuero reiterar a import\xc3\xa2ncia da mudan\xc3\xa7a para todos os cidad\xc3\xa3os do Brasil. \xc3\x89 hora de tr'
Target: b'Discurso 1: \nQuero reiterar a import\xc3\xa2ncia da mudan\xc3\xa7a para todos os cidad\xc3\xa3os do Brasil. \xc3\x89 hora de tra'


In [22]:
# Configurando os lotes de treinamento
BATCH_SIZE = 256
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(256, 100), dtype=tf.int64, name=None), TensorSpec(shape=(256, 100), dtype=tf.int64, name=None))>

In [23]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 1024

# Number of RNN units
rnn_units = 4096

In [24]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x


In [25]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [26]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(256, 100, 97) # (batch_size, sequence_length, vocab_size)


In [27]:
# Verificando o resumo do modelo
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  99328     
                                                                 
 gru (GRU)                   multiple                  62939136  
                                                                 
 dense (Dense)               multiple                  397409    
                                                                 
Total params: 63435873 (241.99 MB)
Trainable params: 63435873 (241.99 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [29]:
sampled_indices

array([74, 63,  7, 27, 16, 32, 27, 84, 76, 84, 78, 96, 86, 20, 35, 86,  6,
       78, 91, 62, 85, 71, 79, 74, 95, 71, 37,  0, 63, 28, 32, 21, 91, 46,
       17, 14, 48, 33, 46, 60,  8, 66, 33, 80, 41,  2, 63, 37, 86, 91, 89,
       18, 18, 37, 54,  4, 89, 40, 76,  1, 61, 54, 85,  0, 63, 20, 95, 29,
       10, 18, 73, 13, 48, 25, 41, 43, 71, 12, 48, 40, 69, 74, 34, 53, 91,
       16, 84, 25, 45, 47, 34, 38,  8, 39, 51, 96, 76, 40, 68, 17],
      dtype=int64)

In [30]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'buscamos a compaix\xc3\xa3o dos pa\xc3\xadses ricos, mas sim solu\xc3\xa7\xc3\xb5es estruturais que fa\xc3\xa7am parte de uma mudan\xc3\xa7a g'

Next Char Predictions:
 b'yn(C4HC\xc3\xa7\xc2\xaa\xc3\xa7\xc3\x89\xe2\x80\x9d\xc3\xaa8K\xc3\xaa%\xc3\x89\xc3\xb6m\xc3\xa9v\xc3\x8dy\xe2\x80\x9cvM[UNK]nDH9\xc3\xb6V52YIVk)qI\xc3\xa0Q nM\xc3\xaa\xc3\xb6\xc3\xb466Me"\xc3\xb4P\xc2\xaa\nle\xc3\xa9[UNK]n8\xe2\x80\x9cE-6x1YAQSv0YPtyJd\xc3\xb64\xc3\xa7AUXJN)Ob\xe2\x80\x9d\xc2\xaaPs5'


In [31]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [32]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (256, 100, 97)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.575513, shape=(), dtype=float32)


In [33]:
tf.exp(example_batch_mean_loss).numpy()

97.07782

In [34]:
#model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)  # Defina a taxa de aprendizado desejada
model.compile(optimizer=optimizer, loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [35]:
# Directory where the checkpoints will be saved
checkpoint_dir = 'C:\\Users\\natha\\anaconda3\\Text Generation with GAN_1'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [36]:
# Treinamento do modelo
EPOCHS = 100
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
4/4 [==============================] - 395s 100s/step - loss: 4.5392 - accuracy: 0.1383
Epoch 2/100
4/4 [==============================] - 354s 89s/step - loss: 4.3612 - accuracy: 0.1568
Epoch 3/100
4/4 [==============================] - 353s 89s/step - loss: 3.9859 - accuracy: 0.1436
Epoch 4/100
4/4 [==============================] - 351s 89s/step - loss: 4.0081 - accuracy: 0.1797
Epoch 5/100
4/4 [==============================] - 361s 91s/step - loss: 4.0015 - accuracy: 0.2297
Epoch 6/100
4/4 [==============================] - 580s 164s/step - loss: 3.8416 - accuracy: 0.1986
Epoch 7/100
4/4 [==============================] - 403s 99s/step - loss: 3.4859 - accuracy: 0.1385
Epoch 8/100
4/4 [==============================] - 353s 89s/step - loss: 3.3396 - accuracy: 0.1292
Epoch 9/100
4/4 [==============================] - 358s 89s/step - loss: 3.1629 - accuracy: 0.1950
Epoch 10/100
4/4 [==============================] - 365s 92s/step - loss: 3.1445 - accuracy: 0.1475
Epoch 1

4/4 [==============================] - 350s 88s/step - loss: 2.1466 - accuracy: 0.3604
Epoch 83/100
4/4 [==============================] - 351s 88s/step - loss: 2.1408 - accuracy: 0.3624
Epoch 84/100
4/4 [==============================] - 347s 87s/step - loss: 2.1365 - accuracy: 0.3632
Epoch 85/100
4/4 [==============================] - 364s 92s/step - loss: 2.1320 - accuracy: 0.3649
Epoch 86/100
4/4 [==============================] - 353s 88s/step - loss: 2.1272 - accuracy: 0.3660
Epoch 87/100
4/4 [==============================] - 363s 90s/step - loss: 2.1233 - accuracy: 0.3671
Epoch 88/100
4/4 [==============================] - 367s 90s/step - loss: 2.1178 - accuracy: 0.3683
Epoch 89/100
4/4 [==============================] - 360s 91s/step - loss: 2.1139 - accuracy: 0.3706
Epoch 90/100
4/4 [==============================] - 359s 91s/step - loss: 2.1090 - accuracy: 0.3723
Epoch 91/100
4/4 [==============================] - 376s 94s/step - loss: 2.1037 - accuracy: 0.3721
Epoch 92/100


In [ ]:
# Carregue o modelo treinado
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
4/4 [==============================] - 371s 94s/step - loss: 2.0578 - accuracy: 0.3876
Epoch 2/100
4/4 [==============================] - 369s 94s/step - loss: 2.0532 - accuracy: 0.3880
Epoch 3/100
4/4 [==============================] - 359s 90s/step - loss: 2.0478 - accuracy: 0.3903
Epoch 4/100
4/4 [==============================] - 371s 92s/step - loss: 2.0427 - accuracy: 0.3919
Epoch 5/100
4/4 [==============================] - 367s 92s/step - loss: 2.0376 - accuracy: 0.3929
Epoch 6/100
4/4 [==============================] - 366s 92s/step - loss: 2.0340 - accuracy: 0.3942
Epoch 7/100
4/4 [==============================] - 365s 93s/step - loss: 2.0289 - accuracy: 0.3959
Epoch 8/100
4/4 [==============================] - 361s 90s/step - loss: 2.0244 - accuracy: 0.3965
Epoch 9/100
4/4 [==============================] - 362s 91s/step - loss: 2.0209 - accuracy: 0.3977
Epoch 10/100
4/4 [==============================] - 370s 94s/step - loss: 2.0166 - accuracy: 0.3999
Epoch 11/

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
import time

start = time.time()
states = None
next_char = tf.constant(['DISCURSO:', 'DISCURSO:', 'DISCURSO:', 'DISCURSO:', 'DISCURSO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [ ]:
states = None
next_char = tf.constant(['DISCURSO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))


Gráfico de Precisão: Se estiver treinando um modelo de classificação, você pode adicionar um gráfico de precisão da mesma maneira que o gráfico de perda.

In [ ]:
import matplotlib.pyplot as plt

# Função para plotar o gráfico de precisão sobre os modelos treinados
def plot_accuracy(history):
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['accuracy'], label='Precisão', marker='o', linestyle='-')
    plt.title('Precisão do Modelo ao Longo das Épocas')
    plt.xlabel('Épocas')
    plt.ylabel('Precisão')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.show()


plot_accuracy(history)

Gráfico de Perda: Isso já está incluído no treinamento, mas você pode personalizá-lo para torná-lo mais visualmente atraente. Você pode usar bibliotecas de plotagem, como Matplotlib ou Seaborn, para criar um gráfico de linha que mostra a perda ao longo das épocas.

In [ ]:
import matplotlib.pyplot as plt

def plot_loss(history):
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Perda', marker='o', linestyle='-')
    plt.title('Perda do Modelo ao Longo das Épocas')
    plt.xlabel('Épocas')
    plt.ylabel('Perda')
    plt.legend()
    plt.show()
    
plot_loss(history)

Comparação de Modelos: Se você deseja comparar o desempenho de diferentes modelos ou configurações de hiperparâmetros, pode criar um gráfico de barras que mostra as métricas (perda, precisão, etc.) de cada modelo lado a lado. Aqui está um exemplo simples usando Matplotlib:

In [ ]:
import matplotlib.pyplot as plt

models = ['Modelo 1', 'Modelo 2', 'Modelo 3']
loss = [0.5, 0.4, 0.6]
accuracy = [0.8, 0.85, 0.75]

fig, ax = plt.subplots(figsize=(10, 6))
width = 0.35
x = range(len(models))

bar1 = ax.bar(x, loss, width, label='Perda')
bar2 = ax.bar([i + width for i in x], accuracy, width, label='Precisão')

ax.set_xlabel('Modelos')
ax.set_ylabel('Métricas')
ax.set_title('Comparação de Modelos')
ax.set_xticks([i + width / 2 for i in x])
ax.set_xticklabels(models)
ax.legend()

plt.show()

Gráfico de Evolução do Texto Gerado: Se você estiver gerando texto com o modelo treinado, pode criar um gráfico que mostre a evolução do texto gerado ao longo das épocas. Por exemplo, você pode escolher algumas épocas intermediárias e gerar texto com o modelo em cada uma delas, depois plotar o texto gerado em um gráfico.

In [ ]:
import matplotlib.pyplot as plt

# Épocas escolhidas
epochs = [10,20,30,40,50,60,70,80,90,100]

# Texto gerado em cada época
generated_text = [
    "Texto gerado na época 10",
    "Texto gerado na época 20",
    "Texto gerado na época 30",
    "Texto gerado na época 40",
    "Texto gerado na época 50",
    "Texto gerado na época 60",
    "Texto gerado na época 70",
    "Texto gerado na época 80",
    "Texto gerado na época 90",
    "Texto gerado na época 100"
]

# Crie um gráfico de linha para mostrar a evolução do texto gerado
plt.figure(figsize=(11, 10))
plt.plot(epochs, generated_text, marker='o', linestyle='-')
plt.title('Evolução do Texto Gerado ao Longo das Épocas')
plt.xlabel('Épocas')
plt.ylabel('Texto Gerado')
plt.grid(True)
plt.show()